In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import nltk
from nltk.corpus import cmudict
nltk.download('cmudict')
import pickle
from HMM_sol import *

[nltk_data] Error loading cmudict: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed
[nltk_data]     (_ssl.c:749)>


In [2]:
# Load in pickled preprocessed data

file = open('../data/sonnets.pkl', 'rb')
sonnets = pickle.load(file)
file.close()

file = open('../data/shakespeare_dics.pkl', 'rb')
dic_to_ids, dic_to_syl, ids_to_dic, syl_to_dic = pickle.load(file)
file.close()

file = open('../data/dic_to_meter.pkl', 'rb')
dic_to_meter = pickle.load(file)
file.close()

file = open('../data/meter_to_dic.pkl', 'rb')
meter_to_dic = pickle.load(file)
file.close()

file = open('../data/shakespeare_rhymes.pkl', 'rb')
rhyme_data_raw = pickle.load(file)
file.close()

# Update rhyming word set to only include words that rhyme, but also end on a stressed syllable
# (these will go at the end of lines)
temp_rhymes = []

for rhyme_set in rhyme_data_raw:
    temp_rhyme_set = []
    for word in rhyme_set:
        if int(dic_to_meter[word][-1]) == 1:
            temp_rhyme_set.append(word)
    temp_rhymes.append(temp_rhyme_set)

rhyme_data_raw = temp_rhymes


In [3]:
import copy

tokenized_sonnet = []

for i in range(len(sonnets)):
    for j in range(len(sonnets[i])):
        for k in range(len(sonnets[i][j])):
            while len(tokenized_sonnet) < i + 1:
                tokenized_sonnet.append([])
            while len(tokenized_sonnet[i]) < j + 1:
                tokenized_sonnet[i].append([])
            while len(tokenized_sonnet[i][j]) < k + 1:
                tokenized_sonnet[i][j].append([])
            
            tokenized_sonnet[i][j][k] = dic_to_ids[sonnets[i][j][k]]
            
tokenized_sonnet = np.array(tokenized_sonnet)


# no don't learn rhyme!

In [4]:
# We first train an HMM to generate sonnets in reverse order
reversed_sonnet = []
for sonnet in tokenized_sonnet:
    for line in sonnet:
        line.reverse()
        temp = line
        reversed_sonnet.append(temp)

In [5]:
# HMM_reversed_20 = unsupervised_HMM(reversed_sonnet, 20, 10)
# with open('../data/HMM_reversed_20.hmm', 'wb+') as f:
#     pickle.dump(HMM_reversed_20, f)

f = open('../data/HMM_reversed_20.hmm', 'rb')
HMM_reversed_20 = pickle.load(f)
f.close()

In [6]:
'''
Generate seven pairs of rhyming words without repetition.
'''
def choose_ending_words(rhyme_data):
    temp = copy.deepcopy(rhyme_data)
    
    index_to_word = {}
    
    pairs = [(0, 2), (1, 3), (4, 6), (5, 7), (8, 10), (9, 11), (12, 13)]
    for i in range(len(pairs)):
        # choosing rhyming scheme per uniform distribution
        choice = np.random.randint(0, len(temp))
        
        # We accidentally picked a word that never rhymes with anything else
        while len(temp[choice]) <= 1:
            choice = np.random.randint(0, len(temp))
        
        first_index, second_index = pairs[i]
        choice1 = random.choice(temp[choice])
        temp[choice].remove(choice1)
        choice2 = random.choice(temp[choice])
        temp[choice].remove(choice2)
        
        index_to_word[first_index] = choice1
        index_to_word[second_index] = choice2
    
    words = []
    for i in range(14):
        words.append(index_to_word[i])
    
    return words

In [7]:
# take first thing that is int thing in txt
meter_data = {}
with open("../data/Syllable_dictionary.txt", "r") as f:
    for line in f:
        temp = line.strip().split(" ")
        word = temp[0]
        count = temp[1:]
        
        for possible_val in count:
            if possible_val.isdigit():
                meter_data[word] = int(possible_val)
                break

In [8]:
# # in the form aabb ccdd eeff gg
end_words = choose_ending_words(rhyme_data_raw)


In [9]:
# Load in pickled preprocessed data

file = open('../data/sonnets.pkl', 'rb')
sonnets = pickle.load(file)
file.close()

file = open('../data/shakespeare_dics.pkl', 'rb')
dic_to_ids, dic_to_syl, ids_to_dic, syl_to_dic = pickle.load(file)
file.close()

file = open('../data/dic_to_meter.pkl', 'rb')
dic_to_meter = pickle.load(file)
file.close()

file = open('../data/meter_to_dic.pkl', 'rb')
meter_to_dic = pickle.load(file)
file.close()

file = open('../data/shakespeare_rhymes.pkl', 'rb')
rhyme_data_raw = pickle.load(file)
file.close()



In [10]:
def generate_emission_sequential(num_syl, dic_to_syl, dic_to_ids, dic_to_meter, A, O, start_word):
    temp = np.array(O)
    most_probable_start_state = np.argmax(temp[:, dic_to_ids[start_word]])
    
    emission = []
    states = []

    # choose starting state
    y_i = most_probable_start_state
    states.append(y_i)
    
    stress = int(dic_to_meter[start_word][0])
        
    syllable_count = dic_to_syl[start_word]
    emission.append(dic_to_ids[start_word])
    
    while syllable_count < num_syl:
        y_i = int(y_i)
        array = range(len(O[y_i]))
        observation_index = np.random.choice(array, p=O[y_i])
        
        while (syllable_count + dic_to_syl[ids_to_dic[observation_index]] > num_syl or
               ((dic_to_syl[ids_to_dic[observation_index]] != 0) and (stress == int(dic_to_meter[ids_to_dic[observation_index]][-1])))):
            observation_index = np.random.choice(array, p=O[y_i])
                    
        emission.append(observation_index)
        syllable_count += dic_to_syl[ids_to_dic[observation_index]]
        
        if dic_to_syl[ids_to_dic[observation_index]] != 0:
            stress = int(dic_to_meter[ids_to_dic[observation_index]][0])
        
    return emission, states



In [12]:
poem = []
for i in range(14):
    start_state = end_words[i]
    emission, states = generate_emission_sequential(10, dic_to_syl, dic_to_ids, dic_to_meter, 
                                                    HMM_reversed_20.A, HMM_reversed_20.O, start_state)
    emission.reverse()
    x = ' '.join([str(ids_to_dic[i]) for i in emission])
    poem.append(x)

In [13]:
for line in poem:
    print(line)

print("\n")    

for line in poem:
    line_syl = []
    
    line_list = line.split()
    
    for word in line_list:
        line_syl.append(dic_to_syl[word])
    print(line_syl, sum(line_syl))
    
print("\n")

for line in poem:
    line_meter = []
    line_list = line.split()
    
    for word in line_list:
        line_meter.append(dic_to_meter[word])
    print(line_meter)
    
    
    

ye dearly comment swerving no growing
sky bind deceived shaken spacious
sets every which feed'st epitaph knowing
quick father me injurious draw gracious
clears devil foes arising rare amiss
wrecked stretched painted thither wrinkles strive
fleece teachest jewel and sky other kiss
cries twofold habitation lie alive
defendant limping gainer beauties used
controlling tomb renewest story bow
cries sight ye gazeth rage defence abused
sky seeing deemed sky deeds directed brow
turn meadows seasons penance sullen bent
untold assemble cherish discontent


[1, 2, 2, 2, 1, 2] 10
[1, 1, 3, 2, 3] 10
[1, 2, 1, 1, 3, 2] 10
[1, 2, 1, 3, 1, 2] 10
[1, 2, 1, 3, 1, 2] 10
[1, 2, 2, 2, 2, 1] 10
[1, 2, 2, 1, 1, 2, 1] 10
[1, 2, 4, 1, 2] 10
[3, 2, 2, 2, 1] 10
[3, 1, 3, 2, 1] 10
[1, 1, 1, 2, 1, 2, 2] 10
[1, 2, 1, 1, 1, 3, 1] 10
[1, 2, 2, 2, 2, 1] 10
[2, 3, 2, 3] 10


['0', '1, 0', '1, 0', '1, 0', '1', '0, 1']
['0', '1', '0, 1, 0', '1, 0', '1, 0, 1']
['0', '1, 0', '1', '0', '1, 0, 1', '0, 1']
['0', '1, 0', '1', 